<a href="https://colab.research.google.com/github/nathsmo/Elements-of-ML/blob/master/Laboratorio_9_EoML_Nathalia_Morales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 9 - Nathalia Morales

En este laboratorio usaremos boosting sobre árboles de decisión y exploraremos cómo optimizar el modelo respecto a los parámetros de boosting (número de árboles, rapidez de aprendizaje y profundidad)

In [0]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

import statsmodels
import statsmodels.formula.api as smf

from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingClassifier

print("Todos los paquetes han sido importados:")

from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp


Todos los paquetes han sido importados:


Trabajaremos sobre la base de datos de casas de Boston de skLearn. Para la implementación del GradientBoosting, puedes referirte a un ejemplo con datos del Titanic en [Medium](https://medium.com/all-things-ai/in-depth-parameter-tuning-for-gradient-boosting-3363992e9bae).



## 1.   Clasifica los precios de casas en categorías ordenadas. Esta nueva variable será tu target.


In [0]:
boston = load_boston()
data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data['PRICE'] = boston.target

In [0]:
data['clas'] = 0

In [0]:
for i in range(0,len(data['PRICE'])):
  if data['PRICE'][i] >= 5 and data['PRICE'][i]< 15:
    data['clas'][i] = 1
  elif data['PRICE'][i] >= 15 and data['PRICE'][i]< 25:
    data['clas'][i] = 2
  elif data['PRICE'][i] >= 25 and data['PRICE'][i]< 35:
    data['clas'][i] = 3 
  elif data['PRICE'][i] >= 35 and data['PRICE'][i]< 45:
    data['clas'][i] = 4
  elif data['PRICE'][i] >= 45 and data['PRICE'][i]< 55:
    data['clas'][i] = 5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set o

In [0]:
data['clas'].unique()
#no deberian de haber 0's

array([2, 3, 4, 1, 5])

In [0]:
data2 = data[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']]
aim = data['clas']

Clases de 1 - 5 por grupos de 10 desde 5 - 14.99

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data2, aim, test_size=0.25)

In [0]:
def evaluate_model(model):

    # fit the model with X_train_scaled, y_train
    model.fit(x_train, y_train)

    # predict the test labels
    prediction = model.predict(x_test)

    # evaluate the model
    train_score = model.score(x_train, y_train)
    test_score = model.score(x_test, y_test)

    print('Train Score: '+ str(train_score))
    print('Test Score:' + str(test_score))
    print('')

    return model

## 2.   Haz el Boosting usando valores para el *learning rate*: 1, 0.5, 0.25, 0.1, 0.05, 0.01. Utiliza alguna medida apropiada para medir la efectividad del modelo en cada caso y busca un valor óptimo. **Explica las ventajas y desventajas de un aprendizaje rápido contra uno lento.**


In [0]:
list_learning_rate = [1, 0.5, 0.25, 0.1, 0.05, 0.01]

for i in list_learning_rate:
  print("Learning rate: ", i)
  model = GradientBoostingClassifier(learning_rate = i)
  trained_model = evaluate_model(model)


Learning rate:  1
Train Score: 1.0
Test Score:0.7165354330708661

Learning rate:  0.5
Train Score: 1.0
Test Score:0.7401574803149606

Learning rate:  0.25
Train Score: 1.0
Test Score:0.7480314960629921

Learning rate:  0.1
Train Score: 1.0
Test Score:0.7559055118110236

Learning rate:  0.05
Train Score: 1.0
Test Score:0.7480314960629921

Learning rate:  0.01
Train Score: 0.9050131926121372
Test Score:0.7244094488188977



* El valor óptimo encontrado es un learning rate de 0.1. Las ventahas de un aprendizaje rápido es el ajuste rápido del modelo de una manera más general que si el parendizaje fuera mas lento y por lo tanto un numero mas bajo. Sin embargo el que el aprendizaje lento puede caer en sobreajuste y bias.

## 3.   Repite el Boosting usando valores para el *number of trees*: 1, 2, 4, 8, 16, 32, 64, 100, 200. Utiliza alguna medida apropiada para medir la efectividad del modelo en cada caso y busca un valor óptimo. **Explica las ventajas y desventajas de usar muchos o pocos árboles**


In [0]:
list_learning_rate2 = [1, 2, 4, 8, 16, 32, 64, 100, 200]

for i in list_learning_rate2:
  print("Number of trees: ", i)
  model = GradientBoostingClassifier(n_estimators=i)
  trained_model = evaluate_model(model)


Number of trees:  1
Train Score: 0.7757255936675461
Test Score:0.6692913385826772

Number of trees:  2
Train Score: 0.8284960422163589
Test Score:0.7007874015748031

Number of trees:  4
Train Score: 0.862796833773087
Test Score:0.7165354330708661

Number of trees:  8
Train Score: 0.8918205804749341
Test Score:0.7244094488188977

Number of trees:  16
Train Score: 0.941952506596306
Test Score:0.7244094488188977

Number of trees:  32
Train Score: 0.9868073878627969
Test Score:0.7322834645669292

Number of trees:  64
Train Score: 1.0
Test Score:0.7480314960629921

Number of trees:  100
Train Score: 1.0
Test Score:0.7559055118110236

Number of trees:  200
Train Score: 1.0
Test Score:0.7401574803149606



* El valor óptimo de números de árboles es 100. Las ventajas de usar muchos arboles es que las estimaciones se adaptan más a los datos pero una desventaja que se pueden sobre-adaptar los datos de entrenamiento y ya no rindan en el test. Las ventajas de usar pocos árboles es que no se sobre-ajustan los datos pero también la data no es procesada lo suficiente para subir el score del modelo.

## 4.   Repite el Boosting usando valores para el *max_depth*: entre 1 y 32, enteros. Utiliza alguna medida apropiada para medir la efectividad del modelo en cada caso y busca un valor óptimo. **Explica las ventajas y desventajas de usar una profundidad muy alta o muy baja**

In [0]:
list_learning_rate3 = range(1, 33)

for i in list_learning_rate3:
  print("Max Depth: ", i)
  model = GradientBoostingClassifier(max_depth = i)
  trained_model = evaluate_model(model)

Max Depth:  1
Train Score: 0.8733509234828496
Test Score:0.6850393700787402

Max Depth:  2
Train Score: 1.0
Test Score:0.7401574803149606

Max Depth:  3
Train Score: 1.0
Test Score:0.7559055118110236

Max Depth:  4
Train Score: 1.0
Test Score:0.7007874015748031

Max Depth:  5
Train Score: 1.0
Test Score:0.7401574803149606

Max Depth:  6
Train Score: 1.0
Test Score:0.7244094488188977

Max Depth:  7
Train Score: 1.0
Test Score:0.7322834645669292

Max Depth:  8
Train Score: 1.0
Test Score:0.7480314960629921

Max Depth:  9
Train Score: 1.0
Test Score:0.7322834645669292

Max Depth:  10
Train Score: 1.0
Test Score:0.7244094488188977

Max Depth:  11
Train Score: 1.0
Test Score:0.7401574803149606

Max Depth:  12
Train Score: 1.0
Test Score:0.7322834645669292

Max Depth:  13
Train Score: 1.0
Test Score:0.7244094488188977

Max Depth:  14
Train Score: 1.0
Test Score:0.7401574803149606

Max Depth:  15
Train Score: 1.0
Test Score:0.7322834645669292

Max Depth:  16
Train Score: 1.0
Test Score:0.7244

* Hubo una mayor efectividad del modelo en la max depth de 3.
Las ventajas que la profundidad maxima sea baja es que los datos no se van a sobreajustan pero también puede que no estemos viendo el minimo global. La ventaja de tener una mayor profundidad es que puede ajustarse más a la data para poder tener mejor calificación en total pero puede sobreajustarse.